In [2]:
import os
import numpy as np
import pandas as pd
import mne
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import FastICA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score



In [3]:
#filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230519_zikken'
filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230529_1_zikken'

brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']

# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65_label1 = list()#label1のデータ
o_data = list()
label_data = list()#labelを結合したデータ


All_data = list()  # 全データを結合

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    #b = np.average(after_data)
    #c = np.var(after_data)
    return after_data

file_name = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name = sorted(file_name)
        file_name.append(file)
print('Filelist')
print(file_name)

Filelist
['EEG_grasping_29-May-2023_sub1-1.mat', 'EEG_grasping_29-May-2023_sub1-2.mat', 'EEG_grasping_29-May-2023_sub2-1.mat', 'EEG_grasping_29-May-2023_sub2-2.mat', 'EEG_grasping_29-May-2023_sub2-3.mat', 'EEG_grasping_29-May-2023_sub2-4.mat', 'EEG_grasping_29-May-2023_sub3-1.mat', 'EEG_grasping_29-May-2023_sub3-2.mat', 'EEG_grasping_29-May-2023_sub3-3.mat', 'EEG_grasping_29-May-2023_sub3-4.mat', 'EEG_grasping_29-May-2023_sub3-5.mat', 'EEG_grasping_29-May-2023_sub3-6.mat']


In [4]:
#解析するデータを選択
file_name1 = [file_name[6],file_name[7],file_name[8],file_name[9],file_name[10],file_name[11]]
file_num1 = len(file_name1)
trial_num = 30
all_trial = len(file_name1) * trial_num
all_d = 0
CH = 64
#label1を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:66, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label1.append(o_data)
print("image data_loading")

image data_loading


In [5]:
c1 = o_data65_label1[0].eeg[: , 0:10240]
for n in range(len(file_name1)*trial_num-1):  # all_trial
    g1 = o_data65_label1[n+1].eeg[: , 0:10240]
    c1 = np.dstack([c1, g1])


In [6]:
all_data = c1.transpose(2,0,1)
print(np.shape(all_data))
all_data = all_data.astype(np.double)

(180, 66, 10240)


In [7]:
filter_bandpass = signal.firwin(numtaps=51,cutoff=[8,32],fs = 1024,pass_zero=False)

In [8]:
for n in range(3):
    for j in range(60):
        for k in range(66):
            #all_data[n*60+j][k]= abs(all_data[n*60+j][k]-np.mean(all_data[n*60+j][0:66],axis=0))
            all_data[n*60+j][k] = signal.lfilter(filter_bandpass,1,all_data[n*60+j][k])
        #all_data[n*60+j] = scipy.stats.zscore(all_data[n*60+j])
    all_data[60*n:60*n+60] = scipy.stats.zscore(all_data[60*n:60*n+60])

In [9]:
# plt.plot(all_data[2][5][1024*6:1024*10])
# plt.show()

In [10]:
label_data = list()#labelを結合したデータ
for n in range(all_trial):
    label_data.append(o_data65_label1[n].label)
print(label_data)
print(np.shape(label_data))

[1, 2, 2, 1, 1, 2, 1, 3, 2, 3, 3, 1, 3, 1, 2, 2, 2, 1, 3, 2, 2, 3, 1, 2, 3, 3, 3, 1, 1, 3, 1, 1, 3, 2, 3, 2, 3, 3, 1, 3, 3, 3, 1, 2, 1, 2, 2, 3, 2, 2, 3, 1, 1, 3, 2, 2, 1, 1, 1, 2, 2, 3, 2, 1, 2, 3, 1, 2, 2, 2, 3, 3, 1, 2, 1, 3, 2, 3, 1, 2, 1, 3, 1, 3, 1, 1, 2, 3, 3, 1, 3, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 3, 2, 1, 3, 2, 3, 3, 1, 1, 3, 3, 2, 3, 3, 1, 2, 2, 1, 1, 3, 3, 2, 3, 1, 2, 1, 3, 2, 2, 3, 3, 1, 3, 1, 2, 2, 1, 1, 1, 3, 3, 1, 2, 3, 2, 2, 1, 2, 1, 2, 2, 3, 2, 1, 3, 1, 3, 2, 1, 3, 3, 3, 2, 1, 1, 2, 3, 2, 3, 1, 1, 3, 2, 2, 3, 1]
(180,)


In [11]:
#numpy配列の定義
task1 = o_data65_label1[0].eeg[: , 0:10240]
task2 = o_data65_label1[0].eeg[: , 0:10240]
task3 = o_data65_label1[0].eeg[: , 0:10240]
for n in range(all_trial):
    if label_data[n] == 1:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task1 = np.dstack([task1, join])
    if label_data[n] == 2:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task2 = np.dstack([task2, join])
    if label_data[n] == 3:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task3 = np.dstack([task3, join])
#task1低速度
task1_data = task1.transpose(2,0,1)
task1_data = task1_data.astype(np.double)
task1_data = np.delete(task1_data,0,axis=0)
print(np.shape(task1_data))
#task2低速度
task2_data = task2.transpose(2,0,1)
task2_data = task2_data.astype(np.double)
task2_data = np.delete(task2_data,0,axis=0)
print(np.shape(task2_data))
#task3想起なし
task3_data = task3.transpose(2,0,1)
task3_data = task3_data.astype(np.double)
task3_data = np.delete(task3_data,0,axis=0)
print(np.shape(task2_data))

(60, 66, 10240)
(60, 66, 10240)
(60, 66, 10240)


In [12]:
processed_task1 = task1_data[:,range(64),1024*6:1024*10]
processed_task2 = task2_data[:,range(64),1024*6:1024*10]
processed_task3 = task3_data[:,range(64),1024*6:1024*10]

In [13]:
class_data = np.concatenate([processed_task1,processed_task3])
class_label = np.zeros(120,dtype=int)
class_label[60:] = 1

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(class_data, class_label, test_size=0.7)

In [ ]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
#csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='average_power',cov_est='epoch')
csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='average_power',cov_est='epoch')
# Use scikit-learn Pipeline with cross_val_score function
n_channels = 64
sampling_freq = 1024  # in Hertz
info = mne.create_info(ch_names=brainAdress, ch_types="eeg", sfreq=sampling_freq)
info.set_montage('biosemi64')

In [16]:
# Printing the results
class_balance = np.mean(class_label == class_label[0])
class_balance = max(class_balance, 1. - class_balance)
#print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),class_balance))

# plot CSP patterns estimated on full data for visualization
for n in range(6):
    CSP_bandpass = signal.firwin(numtaps=51,cutoff=[n*4+8,n*4+12],fs = 1024,pass_zero=False)
    filtered_csp_train = signal.lfilter(CSP_bandpass,1,X_train)
    filtered_csp_test = signal.lfilter(CSP_bandpass,1,X_test)
    tmp_train = csp.fit_transform(filtered_csp_train, Y_train)
    tmp_test = csp.transform(filtered_csp_test)
    if n==0:
        features_train = tmp_train
        features_test = tmp_test
    else:
        features_train=np.concatenate((features_train,tmp_train),axis=1)
        features_test=np.concatenate((features_test,tmp_test),axis=1)
print(features_train.shape)
print(features_test.shape)

Computing rank from data with rank=None
    Using tolerance 1.1e+05 (2.2e-16 eps * 64 dim * 7.6e+18  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.4e+05 (2.2e-16 eps * 64 dim * 1e+19  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.4e+05 (2.2e-16 eps * 64 dim * 9.5e+18  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.7e+05 (2.2e-16 eps * 64 dim * 1.2e+19  max singular value)
  

In [17]:
class_data_cspfit2 = np.reshape(features_test,(84,24*4096))

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(class_data_cspfit2,Y_test, test_size=0.2,random_state=42)

In [18]:
model_gs_cv = MLPClassifier()
random_search = {'hidden_layer_sizes': [100, 1000],
               'max_iter': [1000,2000],
               'random_state': [42], 
               'learning_rate_init':[0.1,  0.001],
               'activation':['identity',  'tanh', 'relu'],
               'solver':['sgd', 'adam', 'lbfgs']}
gscv = GridSearchCV(model_gs_cv, random_search, cv=3, verbose=2)
gscv.fit(X_train2, Y_train2)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=1000, random_state=42, solver=sgd; total time=   2.1s
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=1000, random_state=42, solver=sgd; total time=   1.8s
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=1000, random_state=42, solver=sgd; total time=   1.8s
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=1000, random_state=42, solver=adam; total time=   2.2s
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=1000, random_state=42, solver=adam; total time=   2.2s
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=1000, random_state=42, solver=adam; total time=   2.2s
[CV] END activation=identity, hidden_layer_sizes=100, learning_rate_init=0.1, max_iter=10

: 

: 

In [ ]:
#正答率が最高だった時とその時のパラメータを取得
bestScore = gscv.best_score_
bestParam = gscv.best_params_

#表示
print(f'{bestParam}のときに正答率が最高で{bestScore}')

In [ ]:
# 正答率が最高のモデルを取得
best = gscv.best_estimator_

#↑で予測し、その正答率を表示
best.score(X_test2, Y_test2)

In [ ]:
results = []
for hidden_layer_sizes in [10, 100, 1000]:
    for solver in ['sgd', 'adam', 'lbfgs']:
        for activation in ['identity', 'logistic', 'tanh', 'relu']:
            for learning_rate_init in [0.1, 0.01, 0.001]:
                clf = MLPClassifier(max_iter=10000,
                       hidden_layer_sizes=(hidden_layer_sizes,), 
                       activation=activation, solver=solver,
                       learning_rate_init=learning_rate_init)
                clf.fit(X_train2, Y_train2)
                score = clf.score(X_test2, Y_test2)
                results.append([hidden_layer_sizes, activation,
                       solver, learning_rate_init, score])

grid_search 

In [ ]:
res = pd.DataFrame([dat for dat in sorted(results, key=lambda f: f[4],
            reverse=True)], columns=["hidden_layer_sizes", 
            "activation", "solver", "learning_rate_init", "score"])

In [ ]:
# display(res)
# # CSV ファイル (employee.csv) として出力
# res.to_csv("result.csv")

In [ ]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5)